In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb

# dist feature engineering


In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
X = pd.DataFrame.from_csv('X_before_stack.csv')
test = pd.DataFrame.from_csv('test_before_stack.csv')
y = pd.read_csv('y.csv')
y = np.array(y['burned'].values)

In [3]:
# del X['cancel_time']
# del test['cancel_time']
# del X['driver_found']
# del test['driver_found']

del X['Unnamed: 0.1']
del test['Unnamed: 0.1']

In [4]:
X['isdist0'] = np.array([1 if x[0] == 0 else 0 for x in X.values])
X['isdistminus1'] = np.array([1 if x[0] == -1 else 0 for x in X.values])

test['isdist0'] = np.array([1 if x[0] == 0 else 0 for x in test.values])
test['isdistminus1'] = np.array([1 if x[0] == -1 else 0 for x in test.values])

In [5]:
X_dists_log = np.log( np.array(X['dist'].values) + np.array([2.]*len(X)) )
test_dists_log = np.log( np.array(test['dist'].values) + np.array([2.]*len(test)) )

In [6]:
X['dist'] = X_dists_log
test['dist'] = test_dists_log

In [7]:
data = pd.read_csv('../train.csv')
testcsv = pd.read_csv('../test.csv')

In [8]:
data.index = np.arange(0, len(data))
testcsv.index = np.arange(0, len(testcsv))

In [9]:
X['lat'] = data['lat']
X['lon'] = data['lon']
test['lat'] = testcsv['lat']
test['lon'] = testcsv['lon']

In [13]:
del X['lat']
del X['lon']
del test['lat']
del test['lon']

# gogogo

In [27]:
params = {} 
params['objective']           = "binary:logistic" 
params['booster']             = "gbtree"
params['eval_metric']         = "auc"
params['eta']                 = 0.06
params['max_depth']           = 7
params['subsample']           = 0.95
params['colsample_bytree']    = 0.95
params['seed']                = 228
params['njobs']               = -1
params['silent']              = 1

In [14]:
bound = int(X.shape[0]*0.9)
x_train, x_val = X[:bound], X[bound:]
y_train, y_val = y[:bound], y[bound:]
 
dtrain = xgb.DMatrix(x_train, y_train)
dval = xgb.DMatrix(x_val, y_val)
watchlist = [(dval,'eval')]

In [28]:
num_trees = 5000
esr = 12

gbm = xgb.train(params, dtrain, num_trees, 
                evals=watchlist, verbose_eval=False, 
                early_stopping_rounds = esr)

In [29]:
print 'TOPCHEG: ', gbm.best_ntree_limit

TOPCHEG:  197


In [30]:
X.columns

Index([u'dist', u'city_id=-1', u'city_id=0', u'city_id=1', u'city_id=2',
       u'city_id=3', u'city_id=4', u'day_of_week=fri', u'day_of_week=mon',
       u'day_of_week=sat',
       ...
       u'cluster_id=92', u'cluster_id=93', u'cluster_id=94', u'cluster_id=95',
       u'cluster_id=96', u'cluster_id=97', u'cluster_id=98', u'cluster_id=99',
       u'isdist0', u'isdistminus1'],
      dtype='object', length=458)

### eta=0.06, subsample=0.95:  381 tree

In [ ]:
dtrain = xgb.DMatrix(X, y)
dtest = xgb.DMatrix(test)

In [ ]:
gbm = xgb.train(params, dtrain, gbm.best_ntree_limit)
gbm.save_model('xgb_feature.model')

In [ ]:
prediction = gbm.predict(dtest, ntree_limit=gbm.best_ntree_limit)

In [ ]:
pd.DataFrame(prediction).to_csv('feature_xgb.csv')

In [ ]:
response = pd.DataFrame()
response["Ids"] = np.arange(test.shape[0])
response["Y_prob"] = prediction
 
response.to_csv('xgb_feature.csv',index=None)